In [1]:
# Otras librerías utilizadas en este ejemplo.

from pathlib import Path

import pandas as pd
import spacy

In [2]:
# Principales objetos de banrep necesarios para ejemplo tópicos.

from banrep.corpus import MiCorpus
from banrep.diagnosticos import calcular_coherencias
from banrep.diagnosticos import docs_topicos, topico_dominante, palabras_probables
from banrep.io import Textos, leer_palabras
from banrep.topicos import crear_ldas
from banrep.utils import crear_directorio

In [3]:
# Inicializar un modelo spacy, en este caso español mediano.

nlp = spacy.load('es_core_news_md')

In [4]:
# Uso de función: leer_palabras
# Definir variables a usar como filtros en la creación de corpus.
# Stopwords ubicadas en hoja "banrep_es" de archivo excel.
# Columna "type" determina el grupo y "word" las palabras.

pathstops = '/Users/tombito/Dropbox/datasets/wordlists/stopwords.xlsx'
palabras = leer_palabras(pathstops, 'banrep_es', col_grupo="type", col_palabras="word")
stops = palabras.get("stopword")

tags = ['NUM', 'PUNCT', 'SYM']
ents = []

filtros = dict(stopwords=stops, postags=tags, entities=ents)

In [5]:
# Uso de función: leer_palabras
# Definir listas de palabras (wordlists) cuya presencia se quiere evaluar en los textos.
# En este ejemplo, listas de palabras que representan "mejora" y "deterioro".
# Palabras ubicadas en columna "word" de hoja BANREP de archivo excel, agrupadas por columna "type".

pathwl = '/Users/tombito/Dropbox/datasets/wordlists/sentimiento.xlsx'
wordlists = leer_palabras(pathwl, 'BANREP', col_grupo="type", col_palabras="word")


# Cuantas palabras en cada grupo...

for tipo in wordlists:
    print(f'{len(wordlists.get(tipo))} palabras en grupo {tipo}')

1899 palabras en grupo deterioro
359 palabras en grupo mejora


In [6]:
# Uso de función: iterar_registros
# Definir e iterar los textos a procesar como documentos del corpus (archivos txt).
# chars=69 implica que líneas de longitud inferior a 70 caracteres serán excluidas.
# parrafos=True implica considerar cada párrafo de cada archivo txt como un documento.

dir_datos = Path('/Users/tombito/Dropbox/datasets/banrep/inflacion/corpus/')

datos = Textos(dir_datos, aleatorio=False, chars=69, parrafos=True)

In [7]:
# Uso de clase: MiCorpus
# Inicializa corpus con la información requerida.
# long=9 implica ignorar frases que tienen menos de 10 tokens.

corpus = MiCorpus(nlp, registros=datos, filtros=filtros, long=9, wordlists=wordlists)

print(corpus)

Corpus con 13790 docs y 6154 palabras únicas.


In [8]:
# Uso de función: crear_ldas
# Ejemplo crea modelos de tópicos para 5, 10, y 15 tópicos.
# Diferentes parámetros para modelos LDA pueden consultarse en documentación Gensim.
# https://radimrehurek.com/gensim/models/ldamodel.html - class gensim.models.ldamodel.LdaModel

n_topicos = (5, 10, 15)
params = dict(passes=5, alpha='auto', eta='auto', random_state=100)

modelos = [lda for lda in crear_ldas(corpus, n_topicos, params)]

In [9]:
# Uso de función: calcular_coherencias
# Calcular Coherence Score de cada modelo para determinar el "mejor".
# Mayor Coherence Score es mejor modelo.

scores = [score for score in calcular_coherencias(modelos, corpus)]

coherencia = pd.DataFrame(dict(n=n_topicos, score=scores))

mejor_score = max(scores)
cual = scores.index(mejor_score)
mejor_n = n_topicos[cual]

print(f"Modelo de {mejor_n} tópicos mejor Coherence Score: {mejor_score}")

Modelo de 10 tópicos mejor Coherence Score: 0.503359601203732


In [10]:
# Uso de función: crear_directorio
# Guardar lo útil: modelos bigramas y trigramas, diccionario, y mejor modelo de tópicos.

# Crear directorio de salida
dir_salida = crear_directorio('topicos')

#Guardar modelos de bigramas y trigramas
corpus.ngrams.get('bigrams').save(str(dir_salida.joinpath('bigrams')))
corpus.ngrams.get('trigrams').save(str(dir_salida.joinpath('trigrams')))

# Guardar diccionario
corpus.id2word.save(str(dir_salida.joinpath('id2word')))

# Guardar modelo de tópicos y su visualización

dirtopic = dir_salida.joinpath(f'{mejor_n:0>2}')
crear_directorio(dirtopic)

modelo = modelos[cual]
modelo.save(str(dirtopic.joinpath('topicos.lda')))

# En análisis de tópicos se suele usar PyLDAvis para visualizar resultados...
# Hay que importar la librería para eso.

import warnings

import pyLDAvis
import pyLDAvis.gensim

# Gráfica LDAvis de tópicos y sus palabras

bow = list(corpus)
with warnings.catch_warnings():
    warnings.simplefilter('ignore')
    vis = pyLDAvis.gensim.prepare(modelo, bow, corpus.id2word, sort_topics=False)

pyLDAvis.save_html(vis, str(dirtopic.joinpath('topicos.html')))

In [11]:
# Uso de función: docs_topicos
# Usemos el mejor modelo para ver...
# Distribución de probabilidad de tópicos en cada documento...

doctopics = docs_topicos(modelo, corpus)

doctopics.tail()

,0,1,2,3,4,5,6,7,8,9
13785,NaN,NaN,0.423192,0.068268,0.117111,0.103366,0.059159,NaN,0.142277,0.075782
13786,NaN,NaN,NaN,0.190481,NaN,0.011347,0.128869,0.066685,0.566242,NaN
13787,NaN,NaN,0.353583,NaN,0.265717,0.278274,NaN,0.077066,NaN,NaN
13788,0.030297,0.020669,0.039342,NaN,0.054283,0.684569,NaN,0.037246,0.127263,NaN
13789,0.068110,NaN,0.371621,NaN,NaN,0.369914,NaN,NaN,0.158497,NaN


In [12]:
# Uso de función: topico_dominante
# Cada tópico es dominante en cuantos documentos?
# Tópico 7 es el de mayor probabilidad en el 15% de los documentos...

dominante = topico_dominante(doctopics)

dominante.head()

,topico,docs
0,7,0.1482
1,4,0.1393
2,5,0.1327
3,0,0.1176
4,3,0.1163


In [13]:
# Uso de función: palabras_probables
# Cuales son las palabras más probables en cada uno de los tópicos dominantes...

dfs = []
for topico in dominante['topico']:
    df = palabras_probables(modelo, topico, n=20)
    dfs.append(df)

palabras = pd.concat(dfs, ignore_index=True)

palabras.head()

,palabra,probabilidad,topico
0,observado,0.007226,7
1,promedio,0.007283,7
2,comportamiento,0.007353,7
3,reducción,0.007504,7
4,tasas,0.007716,7


In [14]:
# Estadísticas básicas de cada documento...
# Ejemplo de funcionalidad implícita en el corpus.

# Cuantas frases y palabras en cada documento, y si cada párrafo es documento su número.

exts = ["archivo", "fuente", "parrafo", "frases", "palabras"]
stats = pd.DataFrame(([doc._.get(ext) for ext in exts] for doc in corpus.docs), columns=exts)

# Cuantas palabras de las listas deterioro y mejora presentes en cada documento...

stats['deterioro'] = [sum([token._.get('deterioro') for token in doc]) for doc in corpus.docs]
stats['mejora'] = [sum([token._.get('mejora') for token in doc]) for doc in corpus.docs]

stats.tail()

,archivo,fuente,parrafo,frases,palabras,deterioro,mejora
13785,2018-09.txt,corpus,189,4,56,1,1
13786,2018-09.txt,corpus,190,2,24,1,0
13787,2018-09.txt,corpus,191,3,45,0,1
13788,2018-09.txt,corpus,192,5,83,4,4
13789,2018-09.txt,corpus,193,1,34,1,0
